In [1]:
from pathlib import Path
import pandas as pd
import constants

In [2]:
df_submissions = pd.read_csv(constants.FORM_RESPONSES_PATH)
df_pins = pd.read_csv(constants.SUBMISSION_PINS_PATH)

In [3]:
df_submissions

,Timestamp,Submission Pin,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10
0,3/25/2025 13:25:50,82535682,10.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3/25/2025 13:26:20,82535682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN
2,3/25/2025 13:26:42,45552596,NaN,82.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3,3/25/2025 13:26:53,1111,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


only keep rows with valid pins

In [4]:
df = df_submissions.merge(df_pins, how='inner', left_on='Submission Pin', right_on='pin')
df

,Timestamp,Submission Pin,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,team,pin
0,3/25/2025 13:25:50,82535682,10.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39,82535682
1,3/25/2025 13:26:20,82535682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN,39,82535682
2,3/25/2025 13:26:42,45552596,NaN,82.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,40,45552596


make a datetime column

In [5]:
df['datetime'] = pd.to_datetime(df['Timestamp'])

put most recent submissions first

In [6]:
df.sort_values(by='datetime', inplace=True, ascending=False)

only keep most recent submission per team

In [7]:
df.drop_duplicates(subset='Submission Pin', keep='first', inplace=True)

grade the submissions

In [8]:
df['score'] = 0

In [9]:
def is_correct_answer(question: int, given_answer: int) -> bool:

    if question == 1:
        return given_answer == 465369
    elif question == 2:
        return given_answer == 82
    elif question == 3:
        return given_answer == 6452
    elif question == 4:
        return given_answer == 27
    elif question == 5:
        return given_answer == 2
    elif question == 6:
        return given_answer == 1
    elif question == 7:
        return given_answer == 7573
    elif question == 8:
        return given_answer >= 40147 and given_answer <= 60220
    elif question == 9:
        return given_answer >= 67338 and given_answer <= 101007
    elif question == 10:
        return given_answer >= 75129 and given_answer <= 112694
    else:
        return False


for question in range(1, 11):
    df['score'] += df.apply(lambda row: is_correct_answer(question, row['Question ' + str(question)]), axis=1)

In [10]:
df

,Timestamp,Submission Pin,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,team,pin,datetime,score
2,3/25/2025 13:26:42,45552596,NaN,82.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,40,45552596,2025-03-25 13:26:42,2
1,3/25/2025 13:26:20,82535682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN,39,82535682,2025-03-25 13:26:20,1


rank winner based on highest score and earliest submission

In [14]:
df.sort_values(by=['score','datetime'], inplace=True, ascending=[False,True])

In [15]:
columns = ['team', 'score'] + [col for col in df.columns if col not in ['team', 'score']]
df = df[columns]

In [16]:
df

,team,score,Timestamp,Submission Pin,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,pin,datetime
2,40,2,3/25/2025 13:26:42,45552596,NaN,82.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,45552596,2025-03-25 13:26:42
1,39,1,3/25/2025 13:26:20,82535682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,NaN,82535682,2025-03-25 13:26:20


In [17]:
df.to_csv(constants.RESULTS_PATH, index=False)